In [1]:

%load_ext autoreload
%autoreload 2

In [3]:
import copy
import matplotlib.pyplot as plt
import os
import sys
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

DIR = "/content/drive/MyDrive/OMSCS/DL/FinalProject"
DATA_DIR = f'{DIR}/data'
NAME = 'cifar10'

sys.path.append(os.path.abspath(DIR))

training_set = torchvision.datasets.CIFAR10(root=DATA_DIR, train=True, download=True, 
                                            transform=transforms.ToTensor())

training_indices, validation_indices, _, _ = train_test_split(
    range(len(training_set)),
    training_set.targets,
    stratify=training_set.targets,
    test_size=0.1,
)
training_split = torch.utils.data.Subset(training_set, training_indices)
validation_split = torch.utils.data.Subset(training_set, validation_indices)

print(f"{len(training_split)} in training set")
print(f"{len(validation_split)} in validation set")


Extracting /content/drive/MyDrive/OMSCS/DL/FinalProject/data/cifar-10-python.tar.gz to /content/drive/MyDrive/OMSCS/DL/FinalProject/data
45000 in training set
5000 in validation set


In [9]:
import team36
from team36.defenses.stat_def import VGG
from team36.training import train, validate

learning_rate = 5e-4
momentum = 5e-1
weight_decay = 1e-1
batch_size = 32
epochs = 3

training_loader = torch.utils.data.DataLoader(training_split, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = torch.utils.data.DataLoader(validation_split, batch_size=100, shuffle=False, num_workers=2)

model = VGG(image_size=32, in_channels=3)
if torch.cuda.is_available():
    model = model.cuda()

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), learning_rate,
                            momentum=momentum, weight_decay=weight_decay)

best = 0.0
best_cm = None
best_model = None
train_accuracy_history = []
train_loss_history = []
validation_accuracy_history = []
validation_loss_history = []
for epoch in range(epochs):
    train_acc, train_loss = train(epoch, training_loader, model, optimizer, criterion)
    train_accuracy_history.append(train_acc)
    train_loss_history.append(train_loss)
    
    acc, cm, loss = validate(epoch, test_loader, model, criterion)
    validation_accuracy_history.append(acc)
    validation_loss_history.append(loss)
    
    print("Epoch {0} | Training accuracy: {1}% | Validation accuracy: {2}%".format(epoch, train_acc, acc))
    
    if acc > best:
        best = acc
        best_cm = cm
        best_model = copy.deepcopy(model)
        
training_curve, = plt.plot(train_accuracy_history, label='training')
validation_curve, = plt.plot(validation_accuracy_history, label='validation')
plt.title('Accuracy Curve')
plt.legend(handles=[training_curve, validation_curve])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

training_curve, = plt.plot(train_loss_history, label='training')
validation_curve, = plt.plot(validation_loss_history, label='validation')
plt.title('Loss Curve')
plt.legend(handles=[training_curve, validation_curve])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

print('Best Validation Acccuracy: {:.4f}'.format(best))

torch.save(best_model.state_dict(), f"{DIR}/checkpoints/{NAME}-vgg.pth")

Epoch: [0][0/1407]	Loss 2.3983 (2.3983)	Prec @1 0.0938 (0.0938)	
Epoch: [0][10/1407]	Loss 2.2464 (2.3108)	Prec @1 0.1562 (0.1733)	
Epoch: [0][20/1407]	Loss 1.9102 (2.2061)	Prec @1 0.3125 (0.2128)	
Epoch: [0][30/1407]	Loss 1.8294 (2.1639)	Prec @1 0.3750 (0.2198)	
Epoch: [0][40/1407]	Loss 2.0055 (2.1286)	Prec @1 0.3125 (0.2393)	
Epoch: [0][50/1407]	Loss 1.8317 (2.0988)	Prec @1 0.3125 (0.2537)	
Epoch: [0][60/1407]	Loss 1.8454 (2.0683)	Prec @1 0.4062 (0.2643)	
Epoch: [0][70/1407]	Loss 1.7851 (2.0293)	Prec @1 0.4688 (0.2821)	
Epoch: [0][80/1407]	Loss 1.8926 (2.0009)	Prec @1 0.3438 (0.2940)	
Epoch: [0][90/1407]	Loss 1.5856 (1.9777)	Prec @1 0.4688 (0.3001)	
Epoch: [0][100/1407]	Loss 1.7829 (1.9635)	Prec @1 0.3750 (0.3041)	
Epoch: [0][110/1407]	Loss 1.7813 (1.9462)	Prec @1 0.3438 (0.3086)	
Epoch: [0][120/1407]	Loss 1.6624 (1.9256)	Prec @1 0.3750 (0.3156)	
Epoch: [0][130/1407]	Loss 1.4105 (1.9024)	Prec @1 0.5000 (0.3251)	
Epoch: [0][140/1407]	Loss 1.5762 (1.8946)	Prec @1 0.4688 (0.3278)	
Epoch:

Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: ignored